In [1]:
import json
import os

from qdrant_client import QdrantClient

from psiking.core.base.schema import TextNode, ImageNode, TableNode, Document
from psiking.core.base.schema import json_to_doc, doc_to_json
from psiking.core.storage.vectorstore.qdrant import QdrantSingleHybridVectorStore

from config import experiment_settings, app_settings
from src.embedder import EmbedderModule
from src.models import PipelineSettings, EmbeddingSettings, InputFile

/opt/miniconda3/envs/psiking/lib/python3.10/site-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)


In [2]:
# Load Embedder
embedding_settings = EmbeddingSettings(
    embedding_model_path=str(os.path.join(experiment_settings.model_weight_dir, "embedding")),
    dense_batch_size=4,
    sparse_batch_size=256
)
embedder = EmbedderModule(embedding_settings)

Loaded Dense Embedder
Loaded Sparse Embedder


In [3]:
qdrant_client = QdrantClient(host="localhost", port=app_settings.qdrant_port)
collection_key = "7a01e949-d100-4632-8b26-644d191b5605"

vector_store = QdrantSingleHybridVectorStore(
    collection_name=collection_key,
    client=qdrant_client
)

In [34]:
# query="대한민국 경제"
query="NGFS 회원기관 명단 국가, 회원기관"
# 82566_1
query = "PF 대주단 협약 금융권, 업권별 협약 신청‧개시 현황 저축은행 새마을금고 23.4월"
query_document = Document(
    nodes=[TextNode(text=query)]
)
query_embedding_output = embedder.run([query_document])

100%|██████████| 1/1 [00:00<00:00,  5.55it/s]


In [35]:
# Hybrid Query
results = vector_store.query(
    mode="dense",
    dense_embedding=query_embedding_output.dense.values[0],
    # sparse_embedding_values=query_embedding_output.sparse.values[0],
    # sparse_embedding_indices=query_embedding_output.sparse.indices[0],
    limit=10
)
# results = vector_store.query(
#     mode="hybrid",
#     dense_embedding=query_embedding_output.dense.values[0],
#     sparse_embedding_values=query_embedding_output.sparse.values[0],
#     sparse_embedding_indices=query_embedding_output.sparse.indices[0],
#     limit=10
# )
print(len(results.points))

10


In [36]:
results.points[0].payload

{'source_id': '530f69df-29f4-47c0-97e5-3423b7d6eda3',
 'source_file': '82566_1.pdf'}

In [37]:
def get_document(collection_key, document_id):
    with open(f"storage/collections/{collection_key}/documents/{document_id}.json", "r") as f:
        document_data = json.load(f)
    return json_to_doc(document_data)

In [39]:
for point in results.points:#[:5]:
    point_id = point.id
    document = get_document(collection_key, point_id)
    # point_chunk = docstore.get([point_id])[0]
    print("{} - score {:.3f}".format(point_id, point.score))
    print(document.id_, document.metadata)
    print(document.nodes)
    # print(point_chunk.metadata)
    # print(type(point_chunk.nodes[0]))
    # print(repr(point_chunk.nodes[0].text[:100]))
    print('-'*30)

6c2561b4-58a9-499d-bc77-66b228723958 - score 0.728
6c2561b4-58a9-499d-bc77-66b228723958 {'title': '[보도자료] PF 연착륙을 위한 全 금융권 ｢대주단 협약｣ 개정', 'source_id': '530f69df-29f4-47c0-97e5-3423b7d6eda3', 'source_file': '82566_1.pdf'}
[TextNode(id_='43b9c56c-73b3-4b5a-b6a0-d85863a3d6bd', metadata={'page_no': 1}, text_type=<TextType.PLAIN: 'plain'>, label=<TextLabel.PLAIN: 'plain'>, resource=MediaResource(data=None, text="* 全  금융권 PF 대주단 협약 : 복수 금융업권이 참여하는 사업장에 적용 개별 업권 PF 대주단 협약 : 단일 업권이 참여하는 사업장에 적용 가능\n시행사가  협약  적용을 신청하는 경우 채권금융기관으로 구성된 자율협의회는 시행사가  제출한  사업정상화  계획  등을  심의하여  공동관리  개시  여부를 결정하고,시행사(차주)가 자율협의회와 체결한 특별약정을 위반하거나 사업장의 정상화 가능성이 희박한 경우 협약에 따른 공동관리 절차를 중단할 수 있습니다.\n｢ PF 대주단 협약 ｣ 추진 상황을 살펴보면, '23.4.24일 협약이 개정·시행된 이후  '24.3월말까지  총  484개  사업장이  협약  적용을  신청(금융권  협약 183개, 개별 업권 협약 301개)하였습니다.\n이중  30개  사업장에  대해서는 사업정상화 가능성이 낮다고 판단되어 공동 관리 절차가 부결되었으며, 공동관리 절차가 개시된 443개 사업장 중에서 99개 사업장은 사업성 저하 등으로 정상적 사업 진행이 어렵다고 판단되어 공동관리 절차가 중단되는 등 PF 대주단 협약을 통해 시장에서 사업장 옥석 가리기가 진행되고 있습니다.\nPF 대주단 협약 신청 개시 현황 ‧